# Fix high dimensional gaussian mixture
> Why does my script break past ~8 dimensions?

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from geomstats.geometry.hyperbolic import Hyperbolic

INFO: Using numpy backend


In [4]:
# Reference code:


def wrapped_normal_mixture(
    num_points: int,
    num_classes: int,
    noise_std: float = 1.0,
    n_dim: int = 2,
    default_coords_type: str = "extrinsic",
    seed: int = None,
) -> np.ndarray:
    """Generate points from a mixture of Gaussians on the hyperboloid"""

    # Set seed
    if seed is not None:
        np.random.seed(seed)

    # Make manifold
    hyp = Hyperbolic(dim=n_dim, default_coords_type=default_coords_type)
    origin = np.array([1.0] + [0.0] * n_dim)

    # Generate random means; parallel transport from origin
    means = np.concatenate(
        [
            np.zeros(shape=(num_classes, 1)),
            np.random.normal(size=(num_classes, n_dim)),
        ],
        axis=1,
    )
    means = hyp.metric.exp(tangent_vec=means, base_point=origin)

    # Generate random covariance matrices
    covs = np.zeros((num_classes, n_dim, n_dim))
    for i in range(num_classes):
        covs[i] = np.random.normal(size=(n_dim, n_dim))
        covs[i] = covs[i] @ covs[i].T
    covs = noise_std * covs

    # Generate random class probabilities
    probs = np.random.uniform(size=num_classes)
    probs = probs / np.sum(probs)

    # First, determine class assignments
    classes = np.random.choice(num_classes, size=num_points, p=probs)

    # Sample the appropriate covariance matrix and make tangent vectors
    vecs = [np.random.multivariate_normal(np.zeros(n_dim), covs[c]) for c in classes]
    tangent_vecs = np.concatenate([np.zeros(shape=(num_points, 1)), vecs], axis=1)

    # Transport each tangent vector to its corresponding mean on the hyperboloid
    tangent_vecs_transported = hyp.metric.parallel_transport(
        tangent_vec=tangent_vecs, base_point=origin, end_point=means[classes]
    )

    # Exponential map to hyperboloid at the class mean
    tangent_vecs_transported = tangent_vecs_transported[~np.isclose(hyp.metric.norm(tangent_vecs_transported), 0)]
    points = hyp.metric.exp(tangent_vec=tangent_vecs_transported, base_point=means[classes])

    return points, classes

In [15]:
n_dim = 64
num_classes = 2
noise_std = 0.1
num_points = 1000

np.random.seed(15)

hyp = Hyperbolic(dim=n_dim, default_coords_type="extrinsic")
origin = np.array([1.0] + [0.0] * n_dim)

# Generate random means; parallel transport from origin
means = np.concatenate(
    [
        np.zeros(shape=(num_classes, 1)),
        np.random.normal(size=(num_classes, n_dim)),
    ],
    axis=1,
)
means = hyp.metric.exp(tangent_vec=means, base_point=origin)

# Generate random covariance matrices
covs = np.zeros((num_classes, n_dim, n_dim))
for i in range(num_classes):
    covs[i] = np.random.normal(size=(n_dim, n_dim))
    covs[i] = covs[i] @ covs[i].T
covs = noise_std * covs

# Generate random class probabilities
probs = np.random.uniform(size=num_classes)
probs = probs / np.sum(probs)

# First, determine class assignments
classes = np.random.choice(num_classes, size=num_points, p=probs)

# Sample the appropriate covariance matrix and make tangent vectors
vecs = [np.random.multivariate_normal(np.zeros(n_dim), covs[c]) for c in classes]
tangent_vecs = np.concatenate([np.zeros(shape=(num_points, 1)), vecs], axis=1)

# Transport each tangent vector to its corresponding mean on the hyperboloid
tangent_vecs_transported = hyp.metric.parallel_transport(
    tangent_vec=tangent_vecs, base_point=origin, end_point=means[classes]
)

# Exponential map to hyperboloid at the class mean
tangent_vecs_transported = tangent_vecs_transported[~np.isclose(hyp.metric.norm(tangent_vecs_transported), 0)]
points = hyp.metric.exp(tangent_vec=tangent_vecs_transported, base_point=means[classes])

ValueError: Cannot project a vector of norm 0. in the Minkowski space to the hyperboloid

In [43]:
(np.sum(tangent_vecs_transported[:, 1:] ** 2, axis=1) - tangent_vecs_transported[:, 0] ** 2).min()

192.90119575336576

In [66]:
for i, (vec, label) in enumerate(zip(tangent_vecs_transported, classes)):
    try:
        hyp.metric.exp(tangent_vec=vec, base_point=means[label])
    except ValueError as e:
        print(e, i, label, hyp.embedding_space.metric.squared_norm(vec))

In [68]:
hyp.metric.exp(tangent_vec=tangent_vecs_transported, base_point=means[classes])

ValueError: Cannot project a vector of norm 0. in the Minkowski space to the hyperboloid

In [63]:
for point, label in zip(tangent_vecs_transported, classes):
    try:
        # hyp.regularize(point)
        hyp.metric.exp(tangent_vec=point, base_point=means[label])
    except Exception as e:
        print(e)

In [51]:
hyp.embedding_space.metric.norm(tangent_vecs_transported[0])

18.65048089240135

In [89]:
# # hyp.embedding_space.metric.squared_norm(tangent_vecs_transported)
import geomstats.backend as gs
import geomstats.algebra_utils as utils
import math

# geomstats.backend.all(hyp.embedding_space.metric.squared_norm(tangent_vecs_transported))

sq_norm_tangent_vec = hyp.embedding_space.metric.squared_norm(tangent_vecs_transported)
sq_norm_tangent_vec = gs.clip(sq_norm_tangent_vec, 0, math.inf)

coef_1 = utils.taylor_exp_even_func(sq_norm_tangent_vec, utils.cosh_close_0, order=5)
coef_2 = utils.taylor_exp_even_func(sq_norm_tangent_vec, utils.sinch_close_0, order=5)

exp = gs.einsum("...,...j->...j", coef_1, means[classes]) + gs.einsum("...,...j->...j", coef_2, tangent_vecs_transported)

In [95]:
(hyp.metric.squared_norm(exp) == 0).sum()

54

In [111]:
get_training_data(64, 15)[0].shape

torch.Size([942, 64])

In [99]:
def bad_points(points, base_points, manifold):
    """Avoid the 'Minkowski norm of 0' error by using this"""
    sq_norm_tangent_vec = manifold.embedding_space.metric.squared_norm(points)
    sq_norm_tangent_vec = gs.clip(sq_norm_tangent_vec, 0, math.inf)

    coef_1 = utils.taylor_exp_even_func(sq_norm_tangent_vec, utils.cosh_close_0, order=5)
    coef_2 = utils.taylor_exp_even_func(sq_norm_tangent_vec, utils.sinch_close_0, order=5)

    exp = gs.einsum("...,...j->...j", coef_1, base_points) + gs.einsum("...,...j->...j", coef_2, points)
    return manifold.metric.squared_norm(exp) == 0

tangent_vecs_transported[~bad_points(tangent_vecs_transported, means[classes], hyp)].shape

(946, 65)

In [100]:
tangent_vecs_transported2 = tangent_vecs_transported[~bad_points(tangent_vecs_transported, means[classes], hyp)]
tangent_vecs_transported2.shape

(946, 65)

In [101]:
tangent_vecs_transported2

array([[-5670.1751055 ,  -400.25542215,  1196.82434557, ...,
          323.11737232,   -92.98165705,    72.12281409],
       [-3037.42596521,   123.74230691,  -131.13113219, ...,
          560.41341165,   149.04898875,   277.49645436],
       [ -233.29444934,     6.48489997,   -11.76087079, ...,
           43.47037884,    10.98657061,    20.94913861],
       ...,
       [-1536.56102285,    57.51152286,   -68.82297809, ...,
          286.56082874,    73.26821937,   141.97004085],
       [ 1172.51912655,   -44.2880561 ,    49.18392844, ...,
         -221.56405413,   -56.07035489,  -113.98474097],
       [-1233.09270212,    52.25547833,   -54.08108901, ...,
          229.73819891,    58.59252182,   115.84326846]])